In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY

import mysql.connector
from mysql.connector import errorcode
from ittutils import ittconnection, get_resampled, get_raw_price, get_raw_volume, classification_dataset_from_ts

In [8]:
db_connection = ittconnection('prodcopy')

transaction_coin = 'ETH'
counter_coin = 0
resample_period = 60

query = (" SELECT * FROM indicator_annpriceclassification WHERE \
    transaction_currency='%s' AND \
    counter_currency=%d \
    and resample_period=%d " ) % (transaction_coin, counter_coin, resample_period)

ai_indicator_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
ai_indicator_df['timestamp'] = pd.to_datetime(ai_indicator_df['timestamp'], unit='s')
ai_indicator_df.index = pd.DatetimeIndex(ai_indicator_df.timestamp)


ai_indicator_df.sort_index(inplace=True)

ai_indicator_df.tail(10)


,id,source,counter_currency,transaction_currency,timestamp,resample_period,predicted_ahead_for,probability_same,probability_up,probability_down,ann_model_id
timestamp,,,,,,,,,,,
2018-04-04 12:00:00,29,0,0,ETH,2018-04-04 12:00:00,60,90,0.428784,0.230954,0.340263,1
2018-04-04 13:00:00,172,0,0,ETH,2018-04-04 13:00:00,60,90,0.433804,0.220208,0.345988,1


In [17]:
query = (" SELECT * FROM signal_signal WHERE \
    resample_period=%d \
    AND `signal`='ANN_Simple' " ) % ( resample_period)

signals_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
signals_df['timestamp'] = pd.to_datetime(signals_df['timestamp'], unit='s')
signals_df.index = pd.DatetimeIndex(signals_df.timestamp)


signals_df.sort_index(inplace=True)



In [18]:
signals_df.tail(20)

,id,created_at,modified_at,UI,subscribers_only,text,source,transaction_currency,signal,trend,...,price_change,volume_btc,volume_btc_change,volume_usdt,volume_usdt_change,sent_at,timestamp,rsi_value,counter_currency,resample_period
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-04-04 13:00:00,101007,2018-04-04 13:10:51.013558,2018-04-04 13:10:51.171853,0,1,,0,BCY,ANN_Simple,-1.0,...,-0.002520,None,None,None,None,1.522847e+09,2018-04-04 13:00:00,None,0,60
2018-04-04 13:00:00,101017,2018-04-04 13:13:57.976826,2018-04-04 13:13:58.072660,0,1,,0,XVC,ANN_Simple,-1.0,...,0.001391,None,None,None,None,1.522848e+09,2018-04-04 13:00:00,None,0,60
2018-04-04 13:00:00,101021,2018-04-04 13:14:55.120045,2018-04-04 13:14:55.220121,0,1,,0,NMC,ANN_Simple,-1.0,...,-0.037286,None,None,None,None,1.522848e+09,2018-04-04 13:00:00,None,0,60
2018-04-04 14:00:00,101028,2018-04-04 14:07:21.091093,2018-04-04 14:07:21.204729,0,1,,0,OMG,ANN_Simple,1.0,...,-0.042852,None,None,None,None,1.522851e+09,2018-04-04 14:00:00,None,0,60
2018-04-04 14:00:00,101029,2018-04-04 14:07:33.153793,2018-04-04 14:07:33.274378,0,1,,0,VRC,ANN_Simple,1.0,...,-0.047576,None,None,None,None,1.522851e+09,2018-04-04 14:00:00,None,0,60
2018-04-04 14:00:00,101039,2018-04-04 14:14:07.166112,2018-04-04 14:14:07.279977,0,1,,0,BLK,ANN_Simple,-1.0,...,-0.024201,None,None,None,None,1.522851e+09,2018-04-04 14:00:00,None,0,60
2018-04-04 14:00:00,101045,2018-04-04 14:16:35.900845,2018-04-04 14:16:36.011265,0,1,,0,HUC,ANN_Simple,-1.0,...,-0.014329,None,None,None,None,1.522851e+09,2018-04-04 14:00:00,None,0,60
